### Uso de la API themoviedb para creacion y uso de bases de datos no relacionales

Garantizamos que los contenedores están levantados

In [1]:
!docker ps

CONTAINER ID   IMAGE              COMMAND                  CREATED          STATUS          PORTS                                                       NAMES
825e9a49f783   cassandra:latest   "docker-entrypoint.s…"   57 seconds ago   Up 56 seconds   7000-7001/tcp, 7199/tcp, 9160/tcp, 0.0.0.0:9042->9042/tcp   cassandra
728d2a047ee0   mongo              "docker-entrypoint.s…"   57 seconds ago   Up 56 seconds   0.0.0.0:27017->27017/tcp                                    mongo
480d35cb8585   neo4j              "tini -g -- /startup…"   57 seconds ago   Up 56 seconds   0.0.0.0:7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp    neo4j


Es necesario activar el environment movies-info

Conectamos la API a Mongo


In [4]:
!python api_mongo_transfer.py

Getting movies id per year
Done
Inserting Movies to mongoDB
Still running...
Inserting Credits to mongoDB
Still running...


Regresa el nombre de top 10 directores con mas peliculas:

In [5]:
from pymongo import MongoClient

# Conectar a tu instancia de MongoDB en Docker
client = MongoClient('mongodb://localhost:27017/')

# Seleccionar la base de datos y la colección
db = client['moviesdb']
credits_collection = db['credits']

# Definir la consulta
pipeline = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$group": {"_id": "$crew.name", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

# Ejecutar la consulta
result = list(credits_collection.aggregate(pipeline))

# Mostrar el resultado
for director in result:
    print(f"{director['_id']}, {director['count']}")



Christopher Nolan, 9
Peter Jackson, 8
Ridley Scott, 8
David Fincher, 8
Tim Burton, 8
Steven Spielberg, 7
Quentin Tarantino, 6
Francis Lawrence, 6
Chris Columbus, 6
John Lasseter, 5


Encontrar el título y año de las películas del género "Comedy" usando la coleccion movies (movies.json):


In [6]:
db = client['moviesdb']
movies_collection = db['movies']

# Definir la consulta
pipeline = [
    {"$sort": {"budget": -1}},
    {"$group": {"_id": {"$year": "$release_date"}, "title": {"$first": "$title"}}},
    {"$sort": {"_id": -1}}
    
]

# Ejecutar la consulta
result = list(movies_collection.aggregate(pipeline))

# Mostrar el resultado
for movie in result:
    print(f"{movie['_id']}, {movie['title']}")



2023, Fast X
2022, Avatar: The Way of Water
2021, Spider-Man: No Way Home
2020, Tenet
2019, Avengers: Endgame
2018, Avengers: Infinity War
2017, Justice League
2016, Captain America: Civil War
2015, Avengers: Age of Ultron
2014, The Hobbit: The Battle of the Five Armies
2013, The Hobbit: The Desolation of Smaug
2012, The Dark Knight Rises
2011, Pirates of the Caribbean: On Stranger Tides
2010, Tangled
2009, Harry Potter and the Half-Blood Prince
2008, The Chronicles of Narnia: Prince Caspian
2007, Pirates of the Caribbean: At World's End
2006, Superman Returns
2005, King Kong
2004, Troy
2003, Terminator 3: Rise of the Machines
2002, Treasure Planet
2001, Pearl Harbor
2000, Dinosaur
1999, The World Is Not Enough
1998, Armageddon
1997, Titanic
1996, The Hunchback of Notre Dame
1995, Die Hard: With a Vengeance
1994, True Lies
1993, Cliffhanger
1992, Batman Returns
1991, Terminator 2: Judgment Day
1990, Die Hard 2


Encontrar el título y año de las películas del género "Comedy"

In [7]:
db = client['moviesdb']
movies_collection = db['movies']

# Definir la consulta
query = {"genres": {"$elemMatch": {"name": "Comedy"}}}
projection = {"title": 1, "release_date": 1}

# Ejecutar la consulta
result = list(movies_collection.find(query, projection))

# Mostrar el resultado
for movie in result:
    print(f"{movie['title']}, {movie['release_date'].year}")

# Cerrar la conexión
client.close()

Pretty Woman, 1990
House Party, 1990
Halfaouine: Boy of the Terraces, 1990
Back to the Future Part III, 1990
Gremlins 2: The New Batch, 1990
Teenage Mutant Ninja Turtles, 1990
The Addams Family, 1991
The Naked Gun 2½: The Smell of Fear, 1991
Hook, 1991
Hot Shots!, 1991
Home Alone 2: Lost in New York, 1992
All Ladies Do It, 1992
Sneakers, 1992
Army of Darkness, 1992
Porco Rosso, 1992
Addams Family Values, 1993
Hocus Pocus, 1993
Groundhog Day, 1993
Dennis the Menace, 1993
Mrs. Doubtfire, 1993
Forrest Gump, 1994
Blank Check, 1994
Blankman, 1994
The Mask, 1994
The Santa Clause, 1994
Chungking Express, 1994
Ace Ventura: Pet Detective, 1994
Baby's Day Out, 1994
Toy Story, 1995
Dilwale Dulhania Le Jayenge, 1995
Casper, 1995
Not Love, Just Frenzy, 1996
Jingle All the Way, 1996
Matilda, 1996
Don't Be a Menace to South Central While Drinking Your Juice in the Hood, 1996
101 Dalmatians, 1996
Tremors 2: Aftershocks, 1996
Mars Attacks!, 1996
The Nutty Professor, 1996
Hercules, 1997
Life Is Beautifu

Hacemos el procesamiento de la base de datos en mongo y la cargamos a Cassandra y a Neo4j

In [8]:
!python mongo_cassandra_transfer.py

Consulta para saber información acerca de las películas que se estrenaron en el año 2019

In [9]:
from cassandra.cluster import Cluster
import pandas as pd


# Conectar a tu instancia de Cassandra en Docker
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Utilizar tu keyspace
session.set_keyspace('mov')

# Ejecutar la primera consulta
query1 = "SELECT title, popularity, budget, revenue FROM movies WHERE release_date >= '2019-01-01' AND release_date <= '2019-12-31' ALLOW FILTERING;"
result1 = session.execute(query1)
df1 = pd.DataFrame(list(result1), columns=result1.column_names)
df1
df1 = df1.sort_values(by=['popularity'], ascending=False)
df1.reset_index(drop=True, inplace=True)
df1[['title', 'popularity']]


,title,popularity
0,The Mongolian Connection,316.188995
1,Uri: The Surgical Strike,245.576996
2,Avengers: Endgame,171.100006
3,Frozen II,140.350998
4,Parasite,127.163002
5,Poseído,124.774002
6,After,119.366997
7,Captain Marvel,104.584999
8,Godzilla: King of the Monsters,103.646004
9,Ford v Ferrari,99.737999


Consulta para saber los géneros de las películas que se estrenaron en 1992

In [12]:
# Conectar a tu instancia de Cassandra en Docker
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Utilizar tu keyspace
session.set_keyspace('mov')

# Ejecutar la primera consulta
query2 = "SELECT title, genres FROM movies WHERE release_date >= '1992-01-01' AND release_date <= '1992-12-31' ALLOW FILTERING;"
result2 = session.execute(query2)

df1 = pd.DataFrame(list(result2), columns=result2.column_names)
df1
df1 = df1.sort_values(by=['title'], ascending=False)
df1.reset_index(drop=True, inplace=True)
df1[['title', 'genres']]

,title,genres
0,Unforgiven,(Western)
1,Trespass,"(Action, Crime, Thriller)"
2,The Bodyguard,"(Action, Drama, Music, Romance, Thriller)"
3,Sneakers,"(Comedy, Crime, Drama)"
4,Scent of a Woman,(Drama)
5,Reservoir Dogs,"(Crime, Thriller)"
6,Porco Rosso,"(Adventure, Animation, Comedy, Family)"
7,Patriot Games,"(Action, Crime, Drama, Thriller)"
8,Home Alone 2: Lost in New York,"(Adventure, Comedy, Family)"
9,Dragon Ball Z: The Return of Cooler,"(Action, Animation, Science Fiction)"


Consulta para saber cuántas personas participaron en la producción de Goodfellas


In [11]:
query3 = "SELECT movie_id, COUNT(*) as num_personas_cast FROM movie_cast WHERE movie_id = 769 ALLOW FILTERING;"
result3 = session.execute(query3)
for row in result3:
    print(row.num_personas_cast)


141


In [ ]:
!python mongo_neo4j_transfer.py

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

# Conexión a la base de datos Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "neoneo4j"

driver = GraphDatabase.driver(uri, auth=(username, password))


Consulta 1: Conocer la cantidad de películas por género


In [ ]:
query_1 = """
MATCH (g:Genre)<-[:IS_GENRE_OF]-(m:Movie)
RETURN g.name, COUNT(m) as moviesCount
ORDER BY moviesCount DESC
"""


Consulta 2: Encontrar películas con conexiones a múltiples géneros


In [ ]:
query_2 = """
MATCH (m:Movie)-[:IS_GENRE_OF]->(g:Genre)
WITH m, COUNT(g) as genreCount
WHERE genreCount > 1
RETURN m.title, genreCount
"""


Consulta 3: Encontrar la película más popular de un género en particular


In [ ]:
query_3 = """
MATCH (g:Genre {name: 'Drama'})<-[:IS_GENRE_OF]-(m:Movie)
WITH m, g ORDER BY m.popularity DESC LIMIT 1
RETURN m.title, m.popularity, g.name as Genre
"""


In [ ]:
with driver.session() as session:
    print("Query 1")
    result_1 = session.run(query_1)
    df_1 = pd.DataFrame(result_1.data(), columns=result_1.keys())
    display(df_1)
    
    print("Query 2")
    result_2 = session.run(query_2)
    df_2 = pd.DataFrame(result_2.data(), columns=result_2.keys())
    display(df_2)

    print("Query 3")
    result_3 = session.run(query_3)
    df_3 = pd.DataFrame(result_3.data(), columns=result_3.keys())
    display(df_3)

# Cerrar la conexión
driver.close()